In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from joblib import dump, load

In [ ]:
import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir)

In [ ]:
random_state = 0

# IMPORT DATASET

In [ ]:
dataset_path = parentdir + "/datasets/EpilepticSeizureRecognition/"

In [ ]:
dataset_name = "EpilepticSeizureRecognition"

In [ ]:
X = pd.read_csv(dataset_path + "data.csv", index_col = 0)
y = np.array(X["y"])
y_all = np.ravel(y).astype("int")
for i in range(2,6):
    y_all[y_all == i] = 2
le = LabelEncoder()
le.fit(y_all)
y_all = le.transform(y_all)
X_all = X.drop("y", axis = 1).values

In [ ]:
from imblearn.under_sampling import RandomUnderSampler # doctest: +NORMALIZE_WHITESPACE
rus = RandomUnderSampler(random_state=random_state, )
X_all, y_all = rus.fit_resample(X_all, y_all)

X_all = X_all.reshape((X_all.shape[0], X_all.shape[1], 1))

In [ ]:
print("X SHAPE: ", X_all.shape)
print("y SHAPE: ", y_all.shape)
unique, counts = np.unique(y_all, return_counts=True)
print("\nCLASSES BALANCE")
for i, label in enumerate(unique):
    print(label, ": ", round(counts[i]/sum(counts), 2))

## DATASET SPLIT

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
# BLACKBOX TRAIN/TEST SETS SPLIT
X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, 
                                                  test_size=0.2, stratify = y_all, random_state=random_state)

# BLACKBOX/EXPLANATION SETS SPLIT
X_train, X_exp, y_train, y_exp = train_test_split(X_train, y_train, 
                                                  test_size=0.3, stratify = y_train, random_state=random_state)

# BLACKBOX TRAIN/VALIDATION SETS SPLIT
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, 
                                                  test_size=0.2, stratify = y_train, random_state=random_state)

# EXPLANATION TRAIN/TEST SETS SPLIT
X_exp_train, X_exp_test, y_exp_train, y_exp_test = train_test_split(X_exp, y_exp, 
                                                                    test_size=0.2, 
                                                                    stratify = y_exp, 
                                                                    random_state=random_state)

# EXPLANATION TRAIN/VALIDATION SETS SPLIT
X_exp_train, X_exp_val, y_exp_train, y_exp_val = train_test_split(X_exp_train, y_exp_train, 
                                                                  test_size=0.2, 
                                                                  stratify = y_exp_train, 
                                                                  random_state=random_state)

print("SHAPES:")
print("BLACKBOX TRAINING SET: ", X_train.shape)
print("BLACKBOX VALIDATION SET: ", X_val.shape)
print("BLACKBOX TEST SET: ", X_test.shape)
print("EXPLANATION TRAINING SET: ", X_exp_train.shape)
print("EXPLANATION VALIDATION SET: ", X_exp_val.shape)
print("EXPLANATION TEST SET: ", X_exp_test.shape)

In [ ]:
n_timesteps, n_outputs, n_features = X_train.shape[1], len(np.unique(y_all)), 1 
print("TIMESTEPS: ", n_timesteps)
print("N. LABELS: ", n_outputs)

# BLACKBOX MODEL

In [ ]:
import keras
import time

In [ ]:
from blackboxes import *

In [ ]:
import blackboxes

In [ ]:
"""import importlib
importlib.reload(blackboxes)"""

## resnet

In [ ]:
blackbox = build_resnet(n_timesteps, n_outputs)

In [ ]:
blackbox.load_weights(parentdir + "/blackbox_checkpoints/EpilepticSeizureRecognition_blackbox_resnet_20200105_233014_best_weights_+0.99_.hdf5")

In [ ]:
resnet = blackbox

## simple CNN

In [ ]:
blackbox = build_simple_CNN(n_timesteps, n_outputs)

In [ ]:
blackbox.load_weights(parentdir + "/blackbox_checkpoints/EpilepticSeizureRecognition_blackbox_simpleCNN_20200105_225722_best_weights_+0.98_.hdf5")

In [ ]:
simplecnn = blackbox

## KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import make_scorer, mean_squared_error, accuracy_score
from sklearn.model_selection import StratifiedKFold, RepeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV

In [ ]:
from joblib import load, dump

In [ ]:
knn = load(parentdir + "/blackbox_checkpoints/EpilepticSeizureRecognition_blackbox_knn_20200105_225631.joblib")

# AUTOENCODER

In [ ]:
# TENSORBOARD
# tensorboard --logdir=/tmp/autoencoder
# http://0.0.0.0:6006
# lsof -i tcp:6006 | grep -v PID | awk '{print $2}' | xargs kill

In [ ]:
import autoencoders
from autoencoders import *
"""import importlib
importlib.reload(autoencoders)"""

## Standard

In [ ]:
params = {'input_shape': (178, 1), 
          'n_blocks': 8, 
          'latent_dim': 30, 
          'encoder_latent_layer_type': 'dense', 
          'encoder_args': {'filters': [2, 4, 8, 16, 32, 64, 128, 256], 
                           'kernel_size': [21, 18, 15, 13, 11, 8, 5, 3], 
                           'padding': 'same', 
                           'activation': 'elu', 
                           'pooling': [1, 1, 1, 1, 1, 1, 1, 1]}}

aut = Autoencoder(**params, verbose = False)
encoder, decoder, autoencoder = aut.build()

In [ ]:
autoencoder.load_weights(parentdir + "/autoencoder_checkpoints/EpilepticSeizureRecognition_autoencoder_20200106_111007_best_weights_+14872.8621_.hdf5")

# GLOBAL VS LOCAL SHAPELET TREE

In [ ]:
# 50 test set
_, X_exp_test_50, _, y_exp_test_50 = train_test_split(X_exp_test, y_exp_test, 
                                                                  test_size=50, 
                                                                  stratify = y_exp_test, 
                                                                  random_state=random_state)

In [ ]:
import agnosticlocalexplainer
from agnosticlocalexplainer import *

from myutils import BlackboxPredictWrapper
import time
from agnosticglobalexplainer import AgnosticGlobalExplainer, save_shapelet_model, load_shapelet_model
from joblib import dump
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, coverage_error
from global_vs_local_surrogate import build_agnostic_local_explainers, print_report, massive_save_agnostic_local_explainers, massive_load_agnostic_local_explainers, get_all_predictions

## RESNET

In [ ]:
autoencoder = autoencoder
encoder = autoencoder.layers[1]
decoder = autoencoder.layers[2]
blackbox = resnet
blackbox_input_dimensions = 3
labels = ["seizure", "no_seizure"]

In [ ]:
max_iter = 50
file_path = parentdir + "/agnostic_explainers/" + dataset_name + "_" + time.strftime("%Y%m%d_%H%M%S")

In [ ]:
import platform
if platform.system() == "Windows":
    os.makedirs(file_path + "/")
else: os.mkdir(file_path + "/")

In [ ]:
%%time
global_surrogate = AgnosticGlobalExplainer(random_state = random_state, 
                                           max_iter = max_iter, 
                                           distance_quantile_threshold = np.array(list(range(1,10)))/10,
                                           labels = labels)
global_surrogate.fit(X_exp_train[:,:,0], blackbox_predict.predict(X_exp_train))

In [ ]:
%%time
agnostic_explainers = build_agnostic_local_explainers(blackbox, 
                               encoder, 
                               decoder, 
                               autoencoder, 
                               X_exp_test_50, 
                               y_exp_test_50,
                               blackbox_input_dimensions = blackbox_input_dimensions,
                               labels = labels,
                               size = 1000,
                               neigh_type = "geneticp",
                               ngen = 10,
                              max_iter=max_iter,
                              random_state = random_state,
                              distance_quantile_threshold = np.array(list(range(1,10)))/10
                               )

In [ ]:
results_df = get_all_predictions(agnostic_explainers, global_surrogate, blackbox_predict, X_exp_test_50, blackbox_predict.predict(X_exp_train), encoder, decoder)
results_df.to_csv(file_path + "/" + "results_df.csv", sep = ";", index = False)
print_report(results_df)

In [ ]:
%%time
save_shapelet_model(global_surrogate, file_path + "/")
massive_save_agnostic_local_explainers(agnostic_explainers, file_path, verbose = True)

In [ ]:
%%time
global_surrogate = load_shapelet_model(file_path + "/")
agnostic_explainers = massive_load_agnostic_local_explainers(file_path, verbose = True)

In [ ]:
# sanity check
results_df_loaded = get_all_predictions(agnostic_explainers, global_surrogate, blackbox_predict, X_exp_test_50, blackbox_predict.predict(X_exp_train), encoder, decoder)
print(sum(results_df_loaded.values != results_df.values))